In [1]:
import os
import sys
sys.path.insert(0, "../..")
import time
from collections import OrderedDict
import spatialpy
spatialpy.__file__
from spatialpy import Solver
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import iplot
import numpy
%load_ext autoreload
%autoreload 2

In [2]:
class All(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return True

class Left(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] >= 0:
            return True
        return False

class Right(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] < 0:
            return True
        return False

In [3]:
class Diffreactdebugtest(spatialpy.Model):

    def __init__(self, nx, ny, model_name="diffreact_debug_test"):
        spatialpy.Model.__init__(self, model_name)

        D_const = 0.01

        # Species
        A = spatialpy.Species(name="A", diffusion_constant=D_const)
        self.add_species([A])

        # Mesh
        self.domain = spatialpy.Domain.create_2D_domain(
            xlim=[-1, 1], ylim=[-1, 1], nx=nx, ny=ny, type_id=1.0, 
            mass=1.0, nu=1.0, fixed=True,  rho0=1.0, c0=1.0, P0=1.0
        )

        # Subdomains
        self.set_type(Left(), 1, mass=1.0, fixed=True)
        self.set_type(Right(), 2, mass=1.0, fixed=True)

        # Parameter
        Rate1 = spatialpy.Parameter("Rate1", expression=100)

        # Reactions
        R1 = spatialpy.Reaction(name="R1", reactants={}, products={A:1}, rate=Rate1, restrict_to=1)
        R2 = spatialpy.Reaction(name="R2", reactants={A:1}, products={}, rate=Rate1, restrict_to=2)

        # Time
        self.timestep_size=0.1
        self.num_timesteps=10
        self.output_freq=1

In [4]:
class CompileTest():

    def __init__(self, max_compile_time, verbose=0):
        self.times = []
        self.num_particles = []
        self.max_compile_time = max_compile_time
        self.verbose = verbose
        
    def build_test(self, nx, ny):
        model = Diffreactdebugtest(nx, ny)
        solver = Solver(model, debug_level=0)
        return model, solver
    
    def run(self):
        self.run_test()
        self.plot_results()
        
    def run_test(self):
        for i in range(2, 10000):
            num_particles = pow(i, 2)
            self.num_particles.append(num_particles)
            
            model, solver = self.build_test(nx=i, ny=i)
            
            start = time.time()
            solver.compile()
            end = time.time()
            diff = end-start
            self.times.append(diff)
            
            if self.verbose >= 1:
                print("The model has {} particles -- Time to compile was {}".format(num_particles, diff))
                
            if diff > self.max_compile_time:
                break
                
    def plot_results(self):
        trace = go.Scatter(
            x=self.num_particles,
            y=self.times,
            mode='lines+markers',
        )
        
        layout = {"title": "Compile Time for Models with Varying Particle Counts",
                  "xaxis_title": "Number of Particles",
                  "yaxis_title": "Average Time (in seconds)"}
        
        fig = {"data":[trace], "layout":layout}

        iplot(fig)

In [ ]:
test = CompileTest(max_compile_time=60)
%time test.run()

In [ ]:
model = Diffreactdebugtest(40, 40)
solver = Solver(model=model)
%prun solver.compile()